# Authentications

In [1]:
# PLEASE USE YOUR INDIVIDUAL GEE ACCOUNT

# !earthengine authenticate 

In [2]:
# # PLEASE USE YOUR INDIVIDUAL GEE ACCOUNT
# # authenticate to Google Colab

# from google.colab import auth
# auth.authenticate_user()

In [3]:
# # USE MIDSCWA@gmail.com/cleanwater
# # to access csv file

# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import time
import ee
ee.Initialize()

# 3. Code

In [5]:
# Sentinel Level 2A surface reflectances
# Note: 2A is a processed file whereby Level 1A top-of-atmosphere TOA 
# reflectance is converted to surface reflectance


# Use the latest Sentinel-2 Cloud Masking with s2cloudless
# https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
# Parameter | Type	| Description
# AOI	ee.Geometry	Area of interest
# START_DATE	string	Image collection start date (inclusive)
# END_DATE	string	Image collection end date (exclusive)
# CLOUD_FILTER	integer	Maximum image cloud cover percent allowed in image 
# collection
# CLD_PRB_THRESH	integer	Cloud probability (%); values greater than are 
# considered cloud
# NIR_DRK_THRESH	float	Near-infrared reflectance; values less than are considered 
# potential cloud shadow
# CLD_PRJ_DIST	float	Maximum distance (km) to search for cloud shadows from 
# cloud edges
# BUFFER	integer	Distance (m) to dilate the edge of cloud-identified objects


START_DATE = '2018-08-01'
END_DATE = '2020-04-01'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

# # Build a Sentinel-2 collection
# # Sentinel-2 surface reflectance and Sentinel-2 cloud probability are two 
# different image collections. Each collection must be filtered similarly 
# (e.g., by date and bounds) and then the two filtered collections must 
# be joined.

# # Define a function to filter the SR and s2cloudless collections according 
# to area of interest and date parameters, then join them on the system:index 
# property. The result is a copy of the SR collection where each image has a 
# new 's2cloudless' property whose value is the corresponding s2cloudless image.

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by 
    # the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [6]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [7]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [8]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [9]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [10]:
def s2_sr_median_func(lat, lon, buffer_m):
  AOI = ee.Geometry.Point([lon, lat])#.buffer(res).bounds()
  s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
  s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())
  return s2_sr_median
  

# Set up bounding box

In [11]:
def square(lat, lon, size):
  crs_proj = "EPSG:4326"  
  return ee.Geometry.Point([lon, lat], proj=crs_proj).buffer(size).bounds()

# SRTM, JRC and NDVI etc

In [12]:
# SRTM for elevation
srtm = ee.Image('USGS/SRTMGL1_003')

In [13]:
# slope of terrain
slope = ee.Terrain.slope(srtm)

In [14]:
# add JRC bands of interest
jrc = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
jrc_bands = jrc.select("seasonality", "transition", "max_extent")\
                .bandNames().getInfo()
jrc = jrc.select(jrc_bands)
jrc.bandNames().getInfo()

['seasonality', 'transition', 'max_extent']

In [15]:
def make_ndvi(image, red='B4', nir='B8'):
  return image.normalizedDifference([nir, red])  

def make_ndwi(image, green='B3', nir='B8'):
  return image.normalizedDifference([green, nir])  


def make_mndvi(image, red='B4', nir='B8'):
  nir = image.select('B8')
  red = image.select('B4')
  aerosols = image.select('B1')  
  mndvi = (nir.subtract(red)
                      .divide(
                          nir.add(red)
                          .subtract(aerosols.add(aerosols))
                          )
                      .rename('mndvi'))
  return mndvi

def make_mndwi(image, green='B3', swir='B11'):
  green = image.select('B3')
  swir = image.select('B11')
  mndwi = (green.subtract(swir)
                .divide(
                    green.add(swir))
                .rename('mndwi'))
  return mndwi

In [16]:
# choose median of mndwi image collection
def make_med_mndwi(lat, lon, buffer_m):
  AOI = ee.Geometry.Point([lon, lat])#.buffer(res).bounds()
  s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
  med_mndwi = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .map(make_mndwi)                            
                             .median()
                             )
  return med_mndwi # returns the image with median mndwi in the date range
  

In [17]:
# choose the max water pixel from mndwi image collection
def s2_sr_greenestpixel_func2(lat, lon, buffer_m):
  AOI = ee.Geometry.Point([lon, lat])#.buffer(res).bounds()
  s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
  s2_sr_greenestpixel = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .map(make_mndwi)                            
                             .qualityMosaic('mndwi')
                             )
  # returns the image with highest mndwi in the date range
  return s2_sr_greenestpixel 
  
  

# Read in Rapanos dataset

In [18]:
# !pip install wheel
# !pip install pandas

In [19]:
!ls

2021_02_23_ExportImagestoGCS.ipynb Data_combined_regular_clean.csv


In [20]:
import pandas as pd
# datapath = "drive/MyDrive/Data/combined_v2.csv"
datapath = "Data_combined_regular_clean.csv"
df = pd.read_csv(datapath, encoding = "ISO-8859-1")

# column name 'index' is conflicting with the native index of dataframe
# hence, creating a new column named "Index"
df["Index"] = df.index + 1

# Set up global variables for Export/Import

In [69]:
# INSERT YOUR BUCKET HERE:
BUCKET = 'pollutemenot-ai'
# FOLDER = 'test_final'
FOLDER = "GEE_images_final2"

# Exporting Functions


In [70]:
def doExport_RBG2(lat, lon, index_danum, band, size=1000):
  image = s2_sr_median_func(lat, lon, size)
  image = image.select('B4', 'B3', 'B2', 'B8')
  imageRGB = image.visualize(**{'bands': ['B4', 'B3', 'B2'], 'max': 9000, 'min': 0.5})

  if size == 1000:
    size_ = "hires"
  else:
    size_ = 'lores'
  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = imageRGB, 
      description = index_danum,
      bucket = BUCKET,
      # fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum  + band + size_,      
      fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum + '_' + band + '_' + size_,            
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      # crs_transform = crs_transform,
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      formatOptions = {
      "cloudOptimized" : True
      }
      )
  task.start()
    # Block until the task completes.
  # print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

In [71]:
def doExport_index2(lat, lon, index_danum, band, size=1000, func=make_ndvi):
  image = s2_sr_median_func(lat, lon, size)
  # image = image.select('B4', 'B3', 'B2', 'B8')

  if size == 1000:
    size_ = "hires"
  elif size == 10000:
    size_ = 'lores'
  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = func(image), 
      description = index_danum + '_' + size_,
      bucket = BUCKET,
      # fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum  + band + size_,            
      fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum + '_' + band + '_' + size_,      
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      # crs_transform = crs_transform,
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      formatOptions = {
      "cloudOptimized" : True
      }
      )
  task.start()
    # Block until the task completes.
  # print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

In [72]:
def doExport_mmndwi(lat, lon, index_danum, band="gmndwi", size=1000, func=None):
  image = make_med_mndwi(lat, lon, size)
  # image = image.select('B4', 'B3', 'B2', 'B8')

  if size == 1000:
    size_ = "hires"
  elif size == 10000:
    size_ = 'lores'
  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = image, 
      description = index_danum + '_' + size_,
      bucket = BUCKET,
      # fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum  + band + size_,            
      fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum + '_' + band + '_' + size_,      
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      # crs_transform = crs_transform,
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      formatOptions = {
      "cloudOptimized" : True
      }
      )
  task.start()
    # Block until the task completes.
  # print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

In [73]:
def doExport_gmndwi(lat, lon, index_danum, band="gmndwi", size=1000, func=None):
  image = s2_sr_greenestpixel_func2(lat, lon, size)
  # image = image.select('B4', 'B3', 'B2', 'B8')

  if size == 1000:
    size_ = "hires"
  elif size == 10000:
    size_ = 'lores'
  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = image, 
      description = index_danum + '_' + size_,
      bucket = BUCKET,
      # fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum  + band + size_,            
      fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum + '_' + band + '_' + size_,      
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      # crs_transform = crs_transform,
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      formatOptions = {
      "cloudOptimized" : True
      }
      )
  task.start()
    # Block until the task completes.
  # print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

In [74]:
def doExport_srtm2(lat, lon, index_danum, band, size=1000, func=None):
  image = ee.Image('USGS/SRTMGL1_003')
  # image = ee.Terrain.slope(srtm)
  if size == 1000:
    size_ = "hires"
  elif size == 10000:
    size_ = 'lores'
  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = image, 
      description = index_danum + '_' + size_,
      bucket = BUCKET,
      # fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum  + band + size_,            
      fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum + '_' + band + '_' + size_,      
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      # crs_transform = crs_transform,
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      formatOptions = {
      "cloudOptimized" : True
      }
      )
  task.start()
    # Block until the task completes.
  # print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

In [75]:
def doExport_slope2(lat, lon, index_danum, band, size=1000, func=None):
  srtm = ee.Image('USGS/SRTMGL1_003')
  image = ee.Terrain.slope(srtm)
  if size == 1000:
    size_ = "hires"
  elif size == 10000:
    size_ = 'lores'
  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = image, 
      description = index_danum + '_' + size_,
      bucket = BUCKET,
      # fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum  + band + size_,            
      fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum + '_' + band + '_' + size_,      
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      # crs_transform = crs_transform,
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      formatOptions = {
      "cloudOptimized" : True
      }
      )
  task.start()
    # Block until the task completes.
  # print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

In [76]:
# srtm = ee.Image('USGS/SRTMGL1_003')
# lat = 30
# lon = -75

# folder = FOLDER
# size = 1
# size_ = "1"
# band = "srtm"
# index_danum = "1"
# task = ee.batch.Export.image.toCloudStorage(
#       image = srtm, 
#       description = "1" + '_' + "1",
#       bucket = BUCKET,
#       # fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum  + band + size_,            
#       fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum + '_' + band + '_' + size_,      
#       region = square(lat, lon, size).getInfo().get('coordinates'),
#       crs = 'EPSG:4326',
#       # crs_transform = crs_transform,
#       dimensions = "256x256",
#       maxPixels = 1E13,
#       fileFormat = "GeoTIFF",
#       formatOptions = {
#       "cloudOptimized" : True
#       }
#       )
# task.start()

In [77]:
# task.status()

In [78]:
def doExport_jrc2(lat, lon, index_danum, band, size=1000, func=None, res='hires'):
  jrc = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
  # jrc_bands = jrc.select("seasonality", "transition", "max_extent")\
                # .bandNames().getInfo()
  if band == "transition":
    jrc = jrc.select("transition")
  elif band == "max_extent":
    jrc = jrc.select("max_extent")
  else:
    jrc = jrc.select("seasonality")

  if size == 1000:
    size_ = "hires"
  elif size == 10000:
    size_ = 'lores'
  folder = FOLDER
  
  task = ee.batch.Export.image.toCloudStorage(
      image = jrc, 
      description = index_danum + '_' + size_,
      bucket = BUCKET,
      # fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum  + band + size_,            
      fileNamePrefix = folder + '/' + size_ + '/' + band + '/' + index_danum + '_' + band + '_' + size_,      
      region = square(lat, lon, size).getInfo().get('coordinates'),
      crs = 'EPSG:4326',
      # crs_transform = crs_transform,
      dimensions = "256x256",
      maxPixels = 1E13,
      fileFormat = "GeoTIFF",
      formatOptions = {
      "cloudOptimized" : True
      }
      )
  task.start()
    # Block until the task completes.
  # print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

In [79]:
import numpy as np

def export_data2(index_danum, lat, lon):
  for size in [1000, 10000]:
    # doExport_RBG2(lat, lon, index_danum, 'RBG', size)
    # doExport_index2(lat, lon, index_danum, 'ndvi', size, make_ndvi)
    # doExport_index2(lat, lon, index_danum, 'ndwi', size, make_ndwi)
    doExport_index2(lat, lon, index_danum, 'mndvi', size, make_mndvi)
    doExport_index2(lat, lon, index_danum, 'mndwi', size, make_mndwi)
    doExport_gmndwi(lat, lon, index_danum, 'gmndwi', size, None)
    # doExport_gmndwi(lat, lon, index_danum, 'mmndwi', size, None)
    doExport_srtm2(lat, lon, index_danum, 'srtm', size, None)
    # doExport_slope2(lat, lon, index_danum, 'slope', size, None)
    if size == 1000:
      doExport_jrc2(lat, lon, index_danum, 'seasonality', size, None)
      doExport_jrc2(lat, lon, index_danum, 'transition', size, None)
    # doExport_jrc2(lat, lon, index_danum, 'max_extent', size, None, hires)


# Exporting Set up

In [80]:
# Assigned begin and end of records for each person
# MADHUKAR: records 1 - 4000
# SHOBHA: records 4000 - 8000
# RADHIKA: records 8000 - 12000
# JOE: 12000 - last

names = ["MADHUKAR", 'SHOBHA', 'RADHIKA', 'JOE']
start = [1, 4000, 8000, 12000]
end = [4000, 8000, 12000, df.shape[0]]
MY_NAME = "MADHUKAR"

start_dict = dict(zip(names, start))
end_dict = dict(zip(names, end))

In [ ]:
# started 7:27am 2/24
# 1-28 done
index_begin = 29
index_end = 300

if index_begin >= start_dict[MY_NAME] and index_end <= end_dict[MY_NAME]:
  for count in range(index_begin, index_end):
    if count == index_begin: print("exporting index =", count)
    da_number = df.iloc[count-1]["da_number"]
    latitude = df.iloc[count-1]["latitude"]
    longitude = df.iloc[count-1]["longitude"]
    index = count
    export_data2(str(index) + '_' + da_number, latitude, longitude)
    print("Done uploading hires and lores of index =", index)
else:
  print("Please ensure the begin and end is within the interval assigned to you")

print("Woohoo all done!")

exporting index = 29
Done uploading hires and lores of index = 29
Done uploading hires and lores of index = 30
Done uploading hires and lores of index = 31
Done uploading hires and lores of index = 32
Done uploading hires and lores of index = 33
Done uploading hires and lores of index = 34
Done uploading hires and lores of index = 35
Done uploading hires and lores of index = 36
Done uploading hires and lores of index = 37
Done uploading hires and lores of index = 38
Done uploading hires and lores of index = 39
Done uploading hires and lores of index = 40
Done uploading hires and lores of index = 41
Done uploading hires and lores of index = 42
Done uploading hires and lores of index = 43
Done uploading hires and lores of index = 44
Done uploading hires and lores of index = 45
Done uploading hires and lores of index = 46
Done uploading hires and lores of index = 47
Done uploading hires and lores of index = 48
Done uploading hires and lores of index = 49


In [ ]:
stop # stope execution here

# SSURGO

In [ ]:
import pandas as pd
ssurgo_path = "/content/drive/MyDrive/GeoSpatialData/SSURGO/muaggatt.zip"
df_s = pd.read_csv(ssurgo_path, compression='zip', header=0, sep='\t', quotechar='"')

In [ ]:
def find_percent_hydric(lat=None, lon=None):
  ssurgo = ee.Image("users/madhukarreddy/gSSURGO")
  pt = ee.Geometry.Point([lon, lat])
  mukey = ssurgo.select('b1').clip(pt).sample(pt).getInfo()["features"][0]['properties']['b1'] 
  hydclprs = df_s[df_s.mukey == mukey]["hydclprs"]
  return int(hydclprs)

# lat = float(df[df.index == 100].latitude)
# lon = float(df[df.index == 100].longitude)
find_percent_hydric(37.4811, -121.9641) # this is known wetland, so should read 100%

# Opening GeoTIFF images

## Download GCS contents to GDrive

In [ ]:
# create relevant folder for download using %cd and %mkdir
# %cd drive/MyDrive/Madhukar/images
# %mkdir /content/drive/MyDrive/Madhukar/images3

# https://philipplies.medium.com/transferring-data-from-google-drive-to-google-cloud-storage-using-google-colab-96e088a8c041
# !gsutil -m cp -r gs://pollutemenot-ai/test3/hires/gmndwi /content/drive/MyDrive/Madhukar/images/test3_1/hires/gmndwi

# !gsutil -m cp -r "gs://pollutemenot-ai/test3/" "/content/drive/MyDrive/Madhukar/images3/"

In [ ]:
# use !pwd to get local path
# dont forget the / at the end!
local_download_path = r"/content/drive/MyDrive/Madhukar/test_final/lores/srtm/"

In [ ]:
import os
from osgeo import gdal
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

fig = plt.figure(figsize=(10,10))
ax = [None] * 3

# https://howtothink.readthedocs.io/en/latest/PvL_H.html
for i in range(1,4):
  ax[i-1] = fig.add_subplot(2, 2, i) 

count = 0
for filename in os.listdir(local_download_path):
    
  if filename.endswith("tif"): 
      print(filename)
      try:
        dataset = gdal.Open(local_download_path+filename, gdal.GA_ReadOnly) 
        # Note GetRasterBand() takes band no. starting from 1 not 0
        band = dataset.GetRasterBand(1)
        arr = band.ReadAsArray()
        colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]  # R -> G -> B correct form
        # colors = [(0, 0, 1), (0, 1, 0), (1, 0, 0)]  # B -> G -> R
        n_bins = [3, 6, 10, 100]  # Discretizes the interpolation into bins
        cmap_name = 'my_list'
        cm = LinearSegmentedColormap.from_list(cmap_name, colors)

        ax[count].imshow(arr, cmap=cm)
        ax[count].set_title(filename.split("_")[-2])
        # plt.imshow(arr)
        # dataset.GetGeoTransform()
        print("({},{})".format(dataset.GetRasterBand(1).XSize, dataset.GetRasterBand(1).YSize))
      except Exception as e:
        print(e)
  count += 1

plt.show()